In [9]:
#pip install transformers datasets accelerate

In [1]:
import pandas as pd
df = pd.read_csv('../preprocess/combined_finetuning_df.csv')
df = df[df["Buffett_statement"] == 1]

df = df.dropna(subset=["Question", "Answer"])  # drop any incomplete rows
print(f"Filtered dataset size: {len(df)}")

Filtered dataset size: 439


In [2]:
from datasets import Dataset

# Convert pandas DataFrame to HF Dataset
dataset = Dataset.from_pandas(df)

# Optionally, split into train/validation
# For a small dataset, you might do e.g. 90% train, 10% val
dataset = dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = dataset["train"]
val_dataset = dataset["test"]

In [1]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)

model_name = "AdaptLLM/finance-chat"
model_name = "oopere/FinChat-XS"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float32)
# ^ Use torch_dtype if model is half-precision or you have GPU memory constraints


In [15]:
def preprocess_function(examples):
    # Build prompt text
    texts = [
        f"Question: {q}\nAnswer: {a}"
        for q, a in zip(examples["Question"], examples["Answer"])
    ]
    
    # Tokenize in a causal manner
    # Note: We do not separate 'input' vs 'label' because in CLM
    # the model learns to predict every next token in the sequence.
    # We'll rely on the standard LM masking in the collator.
    tokenized = tokenizer(
        texts,
        truncation=True,
        max_length=512,  # adjust as needed
        return_special_tokens_mask=True
    )
    return tokenized

train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/395 [00:00<?, ? examples/s]

Map:   0%|          | 0/44 [00:00<?, ? examples/s]

In [16]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, 
    mlm=False  # Because this is Causal LM, not Masked LM
)


# Training Model

In [ ]:
training_args = TrainingArguments(
    output_dir="oopere-FinChat-XS",
    per_device_train_batch_size=1,  # adjust to fit GPU
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,  # accumulate grads to simulate bigger batch
    num_train_epochs=3,
    evaluation_strategy="steps",
    eval_steps=100,
    logging_steps=50,
    save_steps=500,
    save_total_limit=2,
    report_to="none",  # or "tensorboard"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()


c:\Users\chewy\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\chewy\AppData\Local\Temp\ipykernel_3888\901397049.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss
100,2.549900,2.700572


TrainOutput(global_step=147, training_loss=2.6107002829207855, metrics={'train_runtime': 4422.5682, 'train_samples_per_second': 0.268, 'train_steps_per_second': 0.033, 'total_flos': 671726644617600.0, 'train_loss': 2.6107002829207855, 'epoch': 2.951898734177215})

In [ ]:
# saving it
output_dir = "oopere-FinChat-XS"

# Save model
model.save_pretrained(output_dir)

print(f"Model saved at {output_dir}")

Model and tokenizer saved at oopere-FinChat-XS


In [ ]:
buffetAI = AutoModelForCausalLM.from_pretrained("oopere-FinChat-XS")

In [18]:
# Let's load the trained model from the output dir if needed
# model = AutoModelForCausalLM.from_pretrained("adaptllm-finance-buffett")

# Directly use model.generate():
def buffett_answer(question, max_new_tokens=256):
    prompt = f"Question: {question}\nAnswer:"
    messages = [
        {"role": "user", "content": prompt}
    ]   
    prompt = tokenizer.apply_chat_template(messages, tokenize=False)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            top_p=0.9,
            temperature=0.7,
            repetition_penalty=1.2,
            eos_token_id=tokenizer.eos_token_id,  # clearly defined stop token
            pad_token_id=tokenizer.eos_token_id   # to handle padding gracefully
        )
    inputs = tokenizer(prompt, return_tensors="pt").to(buffetAI.device)
    with torch.no_grad():
        buffet_output_ids = buffetAI.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            top_p=0.9,
            temperature=0.7,
            repetition_penalty=1.2,
            eos_token_id=tokenizer.eos_token_id,  # clearly defined stop token
            pad_token_id=tokenizer.eos_token_id   # to handle padding gracefully
        )
    return tokenizer.decode(output_ids[0], skip_special_tokens=True), tokenizer.decode(buffet_output_ids[0], skip_special_tokens=True)

# Try it
test_q = "How should I think about investing during a recession?"
test_q = "Do you have any investing tips?"
response, buffet_response = buffett_answer(test_q)
print("-"* 10 + "Normal Response" + "-"*10)
print(response)
print("-"* 10 + "BuffetAI Response" + "-"*10)
print(buffet_response)
# "Question: How should I think about investing during a recession? 
#  Answer: I focus on companies with strong fundamentals..."


----------Normal Response----------
system
You are a helpful AI assistant named SmolLM, trained by Hugging Face
user
Question: Do you have any investing tips?
Answer:
assistant
Yes. Investing is the key to financial growth and wealth creation. It involves diversifying your portfolio across various asset classes such as stocks bonds etc.; building an emergency fund; saving for long-term goals like retirement savings; using tax strategies that maximize returns on investment; managing risk through diversification; maintaining discipline in one’s investments (e.g., avoid taking excessive risks); understanding Fusion of market conditions when making decisions about which assets will perform best during certain periods; avoiding bad timing with investment strategy or advice from others who may be mistaken; minimizing fees associated with brokerage accounts. Maintaining patience while conducting research also helps make more informed decisions about what types of funds/assets to invest into; 